<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/chimney_detector.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/chimney_detector.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**
BRAILS++ is not yet available on PyPi. The following line installs the latest version from the GitHub repository using `pip`.

In [1]:
pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

  Cloning https://github.com/NHERI-SimCenter/BrailsPlusPlus to /tmp/pip-req-build-6s_qdstc
  Running command git clone --filter=blob:none --quiet https://github.com/NHERI-SimCenter/BrailsPlusPlus /tmp/pip-req-build-6s_qdstc
  Resolved https://github.com/NHERI-SimCenter/BrailsPlusPlus to commit 5fb904699fc635595d85a27b61f0748ecc584021
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-req-build-6s_qdstc/setup.py", line 4, in <module>
          import brails
        File "/tmp/pip-req-build-6s_qdstc/brails/__init__.py", line 44, in <module>
          from .utils.importer import Importer
        File "/tmp/pip-req-build-6s_qdstc/brails/utils/__init__.py", line 44, in <module>
          from .g

## **Import ImageSet and Importer Methods of BRAILS++**
BRAILS++ offers various Artificial Intelligence-powered modules for predicting building and infrastructure attributes from images, such as the `ChimneyDetector` demonstrated in this example.

The `Importer` class provides the main methods to load these modules. To gather the images needed for attribute prediction, users can use the ImageSet class or BRAILS++ scrapers, which facilitate automated asset geometry and image retrieval. This example illustrates both methods for sourcing imagery for use with the `ChimneyDetector` module.

In [2]:
from brails import Importer
from brails.types.image_set import ImageSet

ModuleNotFoundError: No module named 'brails'

## **Load Street-Level Imagery: (Option 1) Download Imagery by Specifying the Region Name**
The `ChimneyDetector` class is created to identify the existence of chimneys in buildings from street-level imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.
1. <strong>Call the `RegionBoundary` class with the necessary details to specify the region of interest.</strong> In this example, street-level imagery of buildings in Atlantic County, NJ, are downloaded. Therefore, the required information includes the `type` of information provided and the `data` itself, which are the `locationName` and `Atlantic County, NJ`.

2. <strong>Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the OpenStreetMap footprint data through the `OSM_FootprintScraper`.</strong> The only information required to initialize `OSM_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft`. The output of `OSM_FootprintScraper` is an `AssetInventory` saved in `atlantic_fp`.

3. <strong>Get a randomly selected `100` building subset of the footprint inventory obtained using `OSM_FootprintScraper`. </strong> This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `atlantic_fp_subset`. For this example, the random seed is set to `40` for reproducibility of the results.

4. <strong>Get the street-level imagery for the selected subset using `get_images` method of `GoogleStreet` module.</strong> `get_images` requires two inputs 1) AssetInventory for which the images will be retrieved (in this example, `atlantic_fp_subset`) and 2) the path to the folder where the retrieved images will be saved, which in this case is `tmp/street/`.


In [ ]:
# Select a region and create its RegionBoundary:
region_data = {"type": "locationName",
               "data": "Atlantic County, NJ"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

osm_fp_class = importer.get_class("OSM_FootprintScraper")
osm_fp_data = {"length": "ft"}
footprint_scraper = osm_fp_class(osm_fp_data)
atlantic_fp = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 20 buildings using a random seed value of 40:
atlantic_fp_subset = atlantic_fp.get_random_sample(100, 40)

# Get aerial imagery for the selected subset using GoogleSatellite:
google_street_class = importer.get_class("GoogleStreet")
google_street = google_street_class()
images_street = google_street.get_images(
    atlantic_fp_subset, 'tmp/street/')

## **Predict Existence of Chimneys Using the ChimneyDetector Module**
`ChimneyDetector` includes a pre-trained EfficientNet_V2_M model that has been trained on a custom dataset of 100,000 images. This model is capable of idnetifying existence of chimneys.

In [ ]:
importer = Importer()
chimney_detector_class = importer.get_class('ChimneyDetector')
chimney_detector = chimney_detector_class()
predictions = chimney_detector.predict(images_street)

## **Load Street Imagery**
The `ChimneyDetector` class is designed to identify roof types of buildings based on aerial imagery. In this example, we will load images from the `examples/image_processor/images/street_easy` directory of BRAILS++ to showcase how to use the `ChimneyDetector`.

To load these images, you first need to initialize an `ImageSet` object, which will be assigned to aerial_images. This object can then load the target images by scanning the specified directory using the set_directory method. The second argument in set_directory is necessary for loading the images; otherwise, the default behavior of `ImageSet` is to simply set the directory to the string value provided in the first argument (in this case, `images/street`).

## **Retrain Default ChimneyDetector Module**

In [ ]:
chimney_detector.retrain(images_street)